
<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI_2024_NLP - Week 1</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: N-gram models </h1>

In [1]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [30]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile

    Returns:
    data (list of lists): each list is a sentence of the text
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [31]:

print("load training set..")
print("\n")
train_data, vocab = load_data("train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [33]:
def remove_rare_words(data, vocab, mincount=1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count

    Returns:
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    data_with_unk = []
    ##########################################################################
    #                      TODO: Implement this function                     #
    ##########################################################################
    for sentence in data:
        new_sentence = []
        for word in sentence:
            if word in vocab and vocab[word] >= mincount:
                new_sentence.append(word)
            else:
                new_sentence.append('<unk>')
        data_with_unk.append(new_sentence)

    return data_with_unk
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################

In [34]:
print("remove rare words")

train_data = remove_rare_words(train_data, vocab, mincount = 2)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])

remove rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


In [58]:
from collections import defaultdict

def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text
    n (int): size of the n-gram

    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    def _build_ngram(data, n):
        counts = defaultdict(lambda: defaultdict(lambda: 0.0))
        for sentence in data:
            sentence = tuple(sentence)
            ##########################################################################
          #                      TODO: Implement this function                     #
          # dict can be indexed by tuples
          # store in the same dict all the ngrams
          # by using the context as a key and the word as a value
          ##########################################################################
            for i in range(len(sentence) - n + 1):
                context = sentence[i:i+n-1]
                word = sentence[i+n-1]
                counts[context][word] += 1
          ##########################################################################
          #                            END OF YOUR CODE                            #
          ##########################################################################
        prob = defaultdict(lambda: defaultdict(lambda: 0.0))
        # Build the probabilities from the counts
        # Be careful with how you normalize!

        for context in counts.keys():
            # p(w | context) = count(context, w)/ count(context)
            ##########################################################################
            #                      TODO: Implement this function                     #
            ##########################################################################

            total_count = sum(counts[context].values())
            for word in counts[context].keys():
                prob[context][word] = counts[context][word] / total_count
            ##########################################################################
            #                            END OF YOUR CODE                            #
            ##########################################################################
        return prob

    prob = defaultdict(lambda: defaultdict(lambda: 0.0))  # Initialization
    for i in range(1, n + 1):
        ngram_prob = _build_ngram(data, i)
        for context in ngram_prob.keys():
            for word in ngram_prob[context].keys():
                prob[context][word] = ngram_prob[context][word]

    return prob

In [45]:
# RUN TO BUILD NGRAM MODEL
n = 10
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)


build ngram model with n =  10


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [46]:
def get_prob(model, context, w):
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): a sentence
    w(string): the word we need to find its probability given the context
    discount (float): discount factor for backoff

    Returns:
    prob (float): probability of this word given the context
    '''
    # code a recursive function over
    # smaller and smaller context
    # to compute the backoff model
    ##########################################################################
    #                      TODO: Implement this function                     #
    ##########################################################################

    # Base case: if context is empty, return the unigram probability
    if len(context) == 0:
        return model.get((), {}).get(w, 0)

    # Convert context to tuple for consistent dictionary key access
    context_tuple = tuple(context)

    # Check if the word is in the given context
    if context_tuple in model and w in model[context_tuple]:
        return model[context_tuple][w]
    else:
        # Recursively call get_prob with a smaller context (backoff)
        shorter_context_prob = get_prob(model, context[1:], w)
        return shorter_context_prob
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################

In [47]:
def perplexity(model, data, n):
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram

    Retunrs:
    perp(float): the perplexity of the model
    '''
    ##########################################################################
    #                      TODO: Implement this function                     #
    ##########################################################################
    # Replace "pass" statement with your code
    l_p_s = 0.0
    t_w = 0

    for sentence in data:
        sentence = ['<s>'] * (n - 1) + sentence + ['</s>']  # Add start and end tokens
        for i in range(n - 1, len(sentence)):
            context = sentence[i - n + 1:i]
            word = sentence[i]
            prob = get_prob(model, context, word)
            if prob > 0:
                l_p_s += math.log(prob, 2)
            else:
                l_p_s += float('-inf')
            t_w += 1

    perplexity = math.pow(2, -l_p_s / t_w)
    return perplexity
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################

In [48]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model, valid_data, n=n))

The perplexity is inf


In [49]:
def get_proba_distrib(model, context):
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and
    their probabilities

    Returns:
    words_and_probs(dic): {word: probability of word given context}
    '''
    # code a recursive function over context
    # to find the longest available ngram
    ##########################################################################
    #                      TODO: Implement this function                     #
    ##########################################################################
    def _get_proba_distrib(context):
        context_tuple = tuple(context)

        if context_tuple in model:
            return model[context_tuple]
        elif len(context) == 0:
            return {}
        else:
            return _get_proba_distrib(context[1:])

    return _get_proba_distrib(context)
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################

In [50]:
def generate(model):
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }

    Returns:
    sentence (list of strings): a sentence sampled according to the language model.
    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    
    #print (model[("<s>")])
    #print (len(model[tuple(sentence)].values()))
    sentence = ["<s>"]

    while sentence[-1] != "</s>" and len(sentence) < 10:
        ##########################################################################
        #                      TODO: Implement this function                     #
        ##########################################################################
        context = sentence[-(len(model) - 1):]
        proba_distrib = get_proba_distrib(model, context)
        
        if not proba_distrib:
            break

        words = list(proba_distrib.keys())
        probs = list(proba_distrib.values())

        next_word = np.random.choice(words, 1, p=probs)[0]
        sentence.append(next_word)
        ##########################################################################
        #                            END OF YOUR CODE                            #
        ##########################################################################
    return sentence
    

In [51]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'he', 'left', 'the', 'money', 'at', 'home.', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.

In [53]:
print("load training set 2..")
print("\n")
train_data2, vocab = load_data("train2.txt")
print(train_data2[0])
print("\n")
print("how :",vocab['how'])
print("load validation set 2")
valid_data2, _ = load_data("valid2.txt")


load training set 2..


['<s>', 'i', 'liked', 'your', 'idea', 'and', 'adopted', 'it', '.', '</s>']


how : 3195
load validation set 2


In [54]:
n = 3

In [55]:
model = build_ngram(train_data2, n)

In [56]:
perplexity(model,valid_data2, n)

23.940639090840747

In [57]:
generate(model)

['<s>', 'i', 'cast', 'my', 'net', 'into', 'the', 'navy', '<unk>', 'our']